#This Section Will clean and stem the works of an email input

In [24]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
import csv
import re
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
porter = PorterStemmer()
lancaster=LancasterStemmer()

def clean_email(email):
    hdrstart = email.find('\n\n')
    email = email[hdrstart:]
    email = email.lower()

    rx = re.compile('<[^<>]+>|\n')
    email= rx.sub(' ', email)

    rx = re.compile('[0-9]+')
    email = rx.sub('number ', email)

    rx = re.compile('(http|https)://[^\s]*')
    email = rx.sub('httpaddr ', email)

    rx = re.compile('[^\s]+@[^\s]+')
    email = rx.sub('emailaddr ', email)

    rx = re.compile('[$]+')
    email = rx.sub('dollar ', email)

    rx = re.compile('[^a-zA-Z0-9 ]')
    email = rx.sub('', email)
    #Tokenise Email
    token_words=word_tokenize(email)
    stem_sentence = []
    
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

def vectorise_email(email):
    d = {}
    with open('vocab.txt') as f:
        for line in f:
            (key, val) = line.split()
            d[key] = int(val)

    vector = np.zeros([1, len(d)], dtype=int)

    email = clean_email(email)

    for word in email.split():
        if word in d:
            vector[0, d[word]] = 1
    return vector

def create_vocab(path, length=0):
    vocab = {}
    file_list = os.listdir(path)
    for file in file_list:
        f = open(path+'/'+file, 'r')
        try:
            email = f.read()
            cleaned_email = clean_email(email)
            for word in cleaned_email.split():
                if word in vocab:
                    vocab[word] += 1
                else:
                    vocab[word] = 1
            f.close()
        except UnicodeDecodeError:
            f.close()

    f = open('vocab.txt', 'w')
    for i in range(length):
        highest_occurance = max(vocab, key=vocab.get)
        f.write(highest_occurance+' '+str(i)+'\n')
        del vocab[highest_occurance]
    f.close()


## Create Data from emails

In [2]:
def create_data_file(paths):
    spam_paths = paths['spam']
    ham_paths = paths['ham']
    data_csv = open('vectorized_data.csv', 'w')
    writer = csv.writer(data_csv, delimiter=',')
    for spam_path in spam_paths:
        files = os.listdir(spam_path)
        for file in files:
            f = open(spam_path+'/'+file, 'r')
            try:
                email = f.read()
                data = vectorise_email(email)
                data = np.append(data, 1)
                writer.writerow(data)
                f.close()
            except UnicodeDecodeError:
                f.close()
    for ham_path in ham_paths:
        files = os.listdir(ham_path)
        for file in files:
            f = open(ham_path+'/'+file, 'r')
            try:
                email = f.read()
                data = vectorise_email(email)
                data = np.append(data, 0)
                writer.writerow(data)
                f.close()
            except UnicodeDecodeError:
                f.close()
    data_csv.close()

## Function to find model hyper paramaters (NOT FINISHED)

In [68]:
def find_model_svm():
    data = pd.read_csv('vectorized_data.csv', header=None)
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

    svc = SVC(kernel='linear')
    svc.fit(X_train, y_train)

    y_pred = svc.predict(X_val)
    print("Validation Data Set")
    print(confusion_matrix(y_val,y_pred))
    print(classification_report(y_val,y_pred))
    print('\n\n\n')

    y_pred_test = svc.predict(X_test)
    print("Testset Data Set")
    print(confusion_matrix(y_test,y_pred_test))
    print(classification_report(y_test,y_pred_test))

    return svc

def predict_spam(svc, email):
    email = clean_email(email)
    email = vectorise_email(email)
    email = pd.DataFrame(email)

    return svc.predict(email)

# Create A vocab List from spam emails and then create a data set from the vocab list

In [3]:
path = 'Spam Data/spam' # Paths to spam emails
create_vocab(path, 2000) # creates vocab list

# Adding paths that have data we are using for the data set
options = {'spam': ['Spam Data/spam'],
           'ham': ['Spam Data/easy_ham_2',
                   'Spam Data/hard_ham']}
create_data_file(options)

In [69]:
svc = find_model_svm()

Validation Data Set
[[289  10]
 [ 11 245]]
              precision    recall  f1-score   support

           0       0.96      0.97      0.96       299
           1       0.96      0.96      0.96       256

    accuracy                           0.96       555
   macro avg       0.96      0.96      0.96       555
weighted avg       0.96      0.96      0.96       555





Testset Data Set
[[297   7]
 [ 11 240]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       304
           1       0.97      0.96      0.96       251

    accuracy                           0.97       555
   macro avg       0.97      0.97      0.97       555
weighted avg       0.97      0.97      0.97       555

[1 0 1 1 0 0 0 0 1 1 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 1 0 0 1 1 0 0 0 1 1 0
 1 0 1 1 1 1 1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 1 0 1 1 0
 1 0 0 0 0 1 1 0 0 1 1 1 0 0 1 0 0 0 1 1 0 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 1
 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 0

In [64]:
email = """From social-admin@linux.ie  Mon Jul 22 19:29:49 2002
Return-Path: <social-admin@linux.ie>
Delivered-To: yyyy@localhost.netnoteinc.com
Received: from localhost (localhost [127.0.0.1])
	by phobos.labs.netnoteinc.com (Postfix) with ESMTP id 8DED4440C8
	for <jm@localhost>; Mon, 22 Jul 2002 14:29:47 -0400 (EDT)
Received: from dogma.slashnull.org [212.17.35.15]
	by localhost with IMAP (fetchmail-5.9.0)
	for jm@localhost (single-drop); Mon, 22 Jul 2002 19:29:47 +0100 (IST)
Received: from webnote.net (mail.webnote.net [193.120.211.219]) by
    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g6MHQK403338 for
    <jm+ilug-social@jmason.org>; Mon, 22 Jul 2002 18:26:20 +0100
Received: from lugh.tuatha.org (root@lugh.tuatha.org [194.125.145.45]) by
    webnote.net (8.9.3/8.9.3) with ESMTP id SAA24473 for
    <jm+ilug-social@jmason.org>; Sat, 20 Jul 2002 18:15:47 +0100
Received: from lugh (root@localhost [127.0.0.1]) by lugh.tuatha.org
    (8.9.3/8.9.3) with ESMTP id SAA26490; Sat, 20 Jul 2002 18:15:21 +0100
Received: from ie.suberic.net (owsla.ie.suberic.net [62.17.162.83]) by
    lugh.tuatha.org (8.9.3/8.9.3) with ESMTP id SAA26451 for <social@linux.ie>;
    Sat, 20 Jul 2002 18:15:14 +0100
X-Authentication-Warning: lugh.tuatha.org: Host owsla.ie.suberic.net
    [62.17.162.83] claimed to be ie.suberic.net
Received: from owsla.ie.suberic.net (owsla [127.0.0.1]) by ie.suberic.net
    (8.11.6/8.11.6) with ESMTP id g6KHFE925191 for <social@linux.ie>;
    Sat, 20 Jul 2002 18:15:14 +0100
Date: Sat, 20 Jul 2002 18:15:12 +0100
To: Rick Moen <rick@linuxmafia.com>
Cc: social@linux.ie
Subject: Re: [ILUG-Social] Completely silent pc
Message-Id: <20020720181512.D23917@ie.suberic.net>
References: <Pine.GSO.4.10.10207192318220.27597-100000@matrix>
    <20020720051950.GE28205@linuxmafia.com>
MIME-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Disposition: inline
User-Agent: Mutt/1.2.5.1i
In-Reply-To: <20020720051950.GE28205@linuxmafia.com>; from
    rick@linuxmafia.com on Fri, Jul 19, 2002 at 10:19:50PM -0700
X-Operating-System: Linux 2.4.18-5 i686
X-GPG-Fingerprint: 9C1D 16F4 11F1 6BD2 933C  048D ACC7 9840 89D0 7646
From: kevin lyda <kevin+dated+1027617313.021fb5@linux.ie>
X-Delivery-Agent: TMDA/0.57
Sender: social-admin@linux.ie
Errors-To: social-admin@linux.ie
X-Mailman-Version: 1.1
Precedence: bulk
List-Id: Irish Linux Users' Group social events <social.linux.ie>
X-Beenthere: social@linux.ie

On Fri, Jul 19, 2002 at 10:19:50PM -0700, Rick Moen wrote:
> Some people strip down old 486 boxes, take out the hard drive, disable
> the fans, and run the thing from just a floppy drive or a CDR you've
> burned for the purpose.

what about using a pcmcia card and a compaq flash card?  or doing an
nfs boot to a server in a noiser part of the house?  (the boiler room
or something like that)

kevin

--
kevin@suberic.net     that a believer is happier than a skeptic is no more to
fork()'ed on 37058400    the point than the fact that a drunken man is happier
meatspace place: home       than a sober one. the happiness of credulity is a
http://ie.suberic.net/~kevin   cheap & dangerous quality -- g.b. shaw

--
Irish Linux Users' Group Social Events: social@linux.ie
http://www.linux.ie/mailman/listinfo/social for (un)subscription information.
List maintainer: listmaster@linux.ie


"""

In [70]:
predict_spam(svc, email)

array([1])